This lab creates a simple chatbot from Llama-2-7b-chat-hf. This is an open source model created by Meta (i.e., Facebook) The model is optimmized for chat.
Gradio is used for the user interface.


Requirements:

It is necessary to add a token from Hugging Face after getting approval from Meta.


**If possible, change runtime to a T4 GPU.**


* Hugging Face Transformers provides us with a straightforward way to use pre-trained models.
* PyTorch does deep learning operations.
* Accelerate optimizes PyTorch operations, especially on a GPU.

In [ ]:
!pip install transformers torch accelerate

* Hugging Face Transformers provides us with a straightforward way to use pre-trained models.
* PyTorch does deep learning operations.
* Accelerate optimizes PyTorch operations, especially on a GPU.

Install the newest Gradio

In [ ]:
!pip install --upgrade gradio

### Loading Model & Tokenizer

Here, we are loading both the Llama model and its associated tokenizer.

The tokenizer converts  text prompts into a sequence of vectors that the model can process.

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-chat-hf

tokenizer = AutoTokenizer.from_pretrained(model, token=True)

### Creating the Llama Pipeline

Hugging Face transformers library provides a pipeline to handle the process of giving prompts to the model and receiving text as output.

We set up a pipeline for text generation.


*Note*: This cell takes a few minutes to run

In [ ]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

We define the `get_response()` function. This is the simplest method, but does not utilize the chat features of the model.





In [ ]:
def get_response(prompt: str):
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
    )
   # print(sequences)
    print("Chatbot:", sequences[0]['generated_text'])



In [ ]:
get_response("Hi, I'm Rich")

In [ ]:
get_response("What's my name?")

**Our function  `get_response() ` basically uses the language model to generate text.**

* There is no conversation history. The basic approach does not account for past interactions, making it less effective for maintaining a coherent conversation.
* It does not use the chat features of the model.


**Chat prompts of Llama 2**

```
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>

{{ user_message }} [/INST]
```

 **We need to build the Prompt**

The parameters:
- `message` is the current message we're sending
- `history` is the history of conversation as a list of tupples `[(user_msg1, bot_msg1), (usr_msg2, bot_msg2), ...]`


In [ ]:
SYSTEM_PROMPT = """<s>[INST] <<SYS>>
You are a helpful bot. Your answers are clear and concise.
<</SYS>>

"""

# Formatting function for message and history
def format_message(message: str, history: list, memory_limit: int = 3):

    """
    Formats the message and history for the Llama model.

    Parameters:
        message (str): Current message to send.
        history (list): Past conversation history.
        memory_limit (int): Limit on how many past interactions to consider.

    Returns:
        Formatted message string
    """
    # always keep len(history) <= memory_limit
    if len(history) > memory_limit:
        history = history[-memory_limit:]

    if len(history) == 0:
        return SYSTEM_PROMPT + f"{message} [/INST]"

    formatted_message = SYSTEM_PROMPT + f"{history[0][0]} [/INST] {history[0][1]} </s>"

    # Handle conversation history
    for user_msg, model_answer in history[1:]:
        formatted_message += f"<s>[INST] {user_msg} [/INST] {model_answer} </s>"

    # Handle the current message
    formatted_message += f"<s>[INST] {message} [/INST]"

    return formatted_message

### Getting Responses

We need the function to generate responses.

In [ ]:
# Generate a response from the Llama model
def get_llama_response(message: str, history: list):

    """
    Generates a conversational response from the Llama model.

    Parameters:
        message (str): User's input message.
        history (list): Past conversation history.

    Returns:
        str: Generated response from the Llama model.
    """
    query = format_message(message, history)
    response = ""

    sequences = llama_pipeline(
        query,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=1024,
    )

    generated_text = sequences[0]['generated_text']
    response = generated_text[len(query):]  # Remove the prompt from the output

    print("Chatbot:", response.strip())
    return response.strip()


In [ ]:
import gradio as gr

gr.ChatInterface(get_llama_response).launch()


### Conclusion

The results  with an open-source model are not as good as GPT-4, but still illustrate the way LLMs work.



Thanks to Kris Ograbek and Ayan Debnath for their videos and blogs.
See also: https://www.gradio.app/guides/creating-a-chatbot-fast